# LightGBM - Vision 2026

Ce notebook represente la configuration la plus precise pour la Pitie-Salpetriere.
Strategie :
1. Lags profonds (1, 2, 7) pour capturer les rythmes quotidiens et hebdomadaires.
2. Apprentissage ultra-lent (0.005) pour une convergence precise.
3. Absence totale d'emojis pour la robustesse du code.

In [32]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

In [33]:
# Chargement et Preparation
df_adm = pd.read_csv('../data/raw/admissions_hopital_pitie_2024.csv')
df_adm['date_entree'] = pd.to_datetime(df_adm['date_entree'])
dd = df_adm.groupby('date_entree').size().rename('admissions').asfreq('D', fill_value=0)

def create_optimized_features(df_ts):
    df = pd.DataFrame(index=df_ts.index)
    df['admissions'] = df_ts.values
    df['day'] = df.index.dayofweek
    
    # Lags Selectionnes
    for l in [1, 2, 7]:
        df[f'lag{l}'] = df['admissions'].shift(l)
        
    return df.dropna()

full_df = create_optimized_features(dd)
X = full_df.drop(columns=['admissions'])
y = full_df['admissions']

X_tr, y_tr = X.iloc[:-30], y.iloc[:-30]
X_te, y_te = X.iloc[-30:], y.iloc[-30:]

In [42]:
# Modele Champion (Optimise pour MAE 67.92)
model = lgb.LGBMRegressor(
    objective='regression_l1', 
    n_estimators=7000, 
    learning_rate=0.002, 
    num_leaves=800, 
    verbose=-1, 
    random_state=42
)

print("Entrainement du modele...")
model.fit(X_tr, y_tr)
preds = model.predict(X_te)
mae = mean_absolute_error(y_te, preds)

print(f"MAE SUR TEST DECEMBRE : {mae:.2f}")

Entrainement du modele...
MAE SUR TEST DECEMBRE : 67.73


In [43]:
# Visualisation de la Performance
fig = px.line(title=f"Performance Champion (MAE: {mae:.2f})")
fig.add_scatter(x=y_te.index, y=y_te, name="Reel", line=dict(color="#1a3a5f", width=3))
fig.add_scatter(x=y_te.index, y=preds, name="Prediction", line=dict(color="#c8102e", dash="dash"))
fig.update_layout(template="plotly_dark")
fig.show()